<a href="https://colab.research.google.com/github/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/chap03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第三章 リカレントニューラルネットワークと自然言語処理

本章では、自然言語処理で主に使われるリカーレントニューラルネットワークの基礎からより応用的で現在最新の研究に近いAttentionやBERTに関する理解を目指して説明していきます。


## 3.1 はじめに

**自然言語**とは、人と人との日常の意思疎通に用いられる自然発生的な言語のことです。簡単にいうと、日本語や英語のような一般に用いられる言語のことを指します。対照的に、プログラミング言語など、ルールが厳格に決められた言語は**形式言語**などと呼ばれます。自然言語をコンピューターに処理させる一連の技術のことを**自然言語処理**と呼びます。

前章では画像認識などといった**コンピュータービジョン**の分野を扱いました。コンピュータービジョンと自然言語処理は何か違うのでしょうか。コンピュータービジョンというのは、コンピューターに目のような機能を持たせる技術と言えます。コンピュータービジョンでは、例えば画像を見て何が写っているかを判断する、というような人間の目が行うことを行います。言い換えると、解析対象は物理的な光です。このような解析は人間だけができるものではなくて、犬とか猫などにもできますし、より小さな動物でも可能なタスクです。もちろん、ある程度複雑なタスクではありますが、ディープラーニングの技術が発達した今となってはそこまで難しすぎるタスクではないと言ってもいいかもしれません。

それに対して、自然言語は基本的に人間以外の動物には理解できません。たとえば、下図の左の鳥の画像のような、動物でも把握できる「実態」に対して、人間が認知するための特別な次元の異なる概念としての「鳥」という言葉を当てているわけです。また、左の赤い丸のような実態を見て、これは「りんご」であると認知するわけですが、同じ人類でも日本語を知らない人にとっては「りんご」という言葉を聞いただけではそれが何を表すかわからないでしょう。

![chap03_cv_vs_nlp](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_cv_vs_nlp.png?raw=true)
<center>図　コンピュータービジョンと自然言語処理</center>

簡単に、コンピュータービジョンや自然言語処理では、そもそもの扱う対象が異なるというところを認識していただければと思います。

話が少し変わりますが、例えば物体Aと物体Bを分類したいとき、何か指標がないとそれら物体を区別することは難しいでしょう。この指標のことを**特徴量** (feature) と呼びます。目で見た物体が持つ特徴というのは、例えば「丸みがある」だとか「角張っている」だとか「赤い」など、形や色などに関連するものであり、直感的にイメージしやすいでしょう。

では自然言語で構成される文章が持つ特徴とはどんなものが考えられるでしょうか。本の文章から著者を推定するようなタスクであれば、よく使う特定の単語があるとか、一文の長さが長いとか、比喩表現を多用するとかいったところに著者のクセ、つまり特徴が現れるかもしれません。この場合は特徴量として単語の出現回数や文章の長さなどを用いると良さそうです。

質問応答のようなタスクではどうでしょう。適切な回答を返すためにはまず質問の内容そのものを理解する必要があります。会話の中で出てきた質問であれば文脈を記憶しておくことも必要そうです。ImageNetデータセットで訓練したニューラルネットワークがコンピュータービジョンの幅広いタスクに対する特徴抽出器として用いられるように、自然言語処理のタスクに対して汎用的に利用できる特徴抽出器はないのでしょうか。



2012年に畳み込みニューラルネットワークのAlexNetがImageNetコンペティションで優勝してから6年の月日が経ち、2018年にBERTという新しいモデルがGoogleによって発表されました。BERTを用いる際にはまず、Wikipediaなどの大規模なコーパスを利用してBERTを事前訓練します。そしてBERTの出力を、他のタスク用に準備したニューラルネットワークの入力として再度訓練します。その結果、従来のモデルでは解くことが難しかった幅広いタスクに対して好成績を修めました。つまり、まるでImageNetで事前訓練したモデルがコンピュータービジョンのタスクに対して汎用的な特徴抽出器として用いられるように、事前訓練したBERTが自然言語処理の汎用特徴抽出器として機能していると言えるでしょう。この衝撃的な結果は研究領域内のみならず、SNSやメディアなどでも大きく取り上げられました。

BERTとは "Bidirectional Encoder Representation from Transformers" の頭文字をとったものです。これを日本語に意訳すると「双方向のTransformerを用いて言語表現をエンコードするモデル」のようになりますが、日本語にしても一見して理解することは難しいのではないかと思います。

まず、このBERTの核をなす「双方向のTransformer」の「Transformer」が何かというと、これはGoogleが開発したニューラル機械翻訳モデルの一種です。機械翻訳には以前はRNN（リカレントニューラルネットワーク、特にLSTMなど）がよく用いられていたのですが、TransformerではRNNを使わずにAttention機構が利用されています。

本節では、BERTの雰囲気を理解していただくことを目標として、自然言語処理とRNN、Attention機構とは何か、Transformer/BERTの概要という内容について3本立てで説明をしていきます。

## 3.2 自然言語処理とは

まず、機械学習を用いた自然言語処理の話をします。先述の通り、自然言語処理とは人間が使う言葉を機械で処理をするというものです。自然言語処理の応用のタスクにどのようなものとして下図のようなものがあります。

![chap03_cv_vs_nlp](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_nlp_applications.png?raw=true)
<center>図　自然言語処理の主要な応用タスク</center>

たとえばわかりやすいところで言うと、ニュース記事があって、その内容が科学技術について書かれたものなのか、スポーツのニュースなのか、ゴシップ系の芸能ニュースであるのかなどを分類したいとします。人手でニュース記事の分類を行う場合にはざっと文章を眺めていくつかの特徴的な単語を見つければ簡単に分類できます。「野球」と「試合」という単語がニュース記事内にあればそれは高い確率でスポーツニュースでしょう。こう言ったことを機械で行うタスクを文章分類と呼びます。

もっと馴染み深いところで言うとGoogle翻訳やDeepLなどが行う日英翻訳などの機械翻訳も自然言語処理のタスクのひとつです。

また、大量の文章を短い文章にまとめる文章要約というタスクもあります。2019年には、Element AIのメンバーらにより公開された文章要約に関する論文 ([On Extractive and Abstractive Neural Document Summarization with Transformer Language Models](https://arxiv.org/abs/1909.03186)) が話題になりました。というのも、論文にはかならず最初に記述されているアブストラクト（要旨）がありますが、その論文のアブストラクト自体を論文で用いられている文章要約の技術を用いて機械的に生成されていたのです。実際その要約は人間が書いたような自然な文章で書かれています。

あとはチャットボットに代表されるような質問応答などの応用タスクがあります。

## 3.3 リカレントニューラルネットワーク

このようなタスクはニューラルネットワークによって解くのが近年のトレンドとなっています。ただし、一口にニューラルネットワークといっても実際はどういった構造のニューラルネットワークを用いれば良いかは自明ではありません。

一般的に、自然言語処理のタスクを普通の全結合ニューラルネットワークで解こうとするとなかなか難しいことが多いです。なぜ難しいのでしょうか。

例えば、英語の文章を日本語に翻訳するタスクを解きたいとしましょう。まず、英語の文章をニューラルネットワークに対して入力します。たとえば下図のように、"Dogs are cute." という文章を入力したら「犬はかわいい。」というような文章が出てきてほしいというような問題設定でニューラルネットワークを訓練していきます。

一般に入力部の長さや出力部の長さはセンテンスごとに異なります。今回の例では、"Dogs are cute." という3つの単語（ピリオドも単語に含めると定義すれば4単語）を入力としていますが、もっと短い文章であったり長い文章であったりすることもあります。そのため、全結合ニューラルネットワークのように固定の長さの入力があって中間層があって固定の長さの出力層があるというような全結合ニューラルネットワークだとなかなか扱いづらいです。

たとえば100個の入力のユニットを用意しておいて、今回の例のように"Dogs are cute." という3つの単語を入力してあとはブランクであるという情報を入れるような仕組みを利用することもできますが、ブランクが多すぎて学習が進みづらいとか、そもそも200個の単語があった場合にはどうなるかというようにどんどん対応ができなくなるため、このようなアプローチは単純にはうまくいかないでしょう。（ただし、後述するBERTなどのモデルでは固定数の入出力ユニットを用いています。）

![chap03_dense_net_nlp_challenge](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_dense_net_nlp_challenge.png?raw=true)
<center>図　全結合ニューラルネットワークの課題</center>

またこの例では、1番目の入力に "dog" という単語があるとそれは「犬」に対応する、といったことがひとまず学習されるでしょう。しかし、"I like the dog." というような別の文章が入力されたとき、"dog" をどう扱って翻訳すればよいかは自明ではありません。全結合ニューラルネットワークでは、同じ単語でも入力位置によって別物として扱われてしまうため、一回学んだことが次に応用できなくなってしまいます。文章は刻一刻と単語が変化するシーケンス（時系列データ）と考えると、時系列データに適したニューラルネットワークを考える必要がありそうです。

そこでよく用いられるのが**リカレントニューラルネットワーク** (Recurrent Neural Network; RNN) です。**再帰型ニューラルネットワーク**などと呼ばれることもあります。時系列の情報を持つデータ、つまり順番によって内容が変わるようなデータというものに対して広く用いられています。

### 3.3.1 リカレントニューラルネットワークの仕組み

リカレントニューラルネットワークとはどういう仕組みで動いているのかについてもう少し詳しく説明していきます。

下図はひとつの全結合ニューラルネットワークだと思ってください。この図では下方から上方に処理が進んでいきます。まず、一番下からの入力を $x^{<1>}$ とします。この文脈では $x^{<t>}\,(t=0,...,T)$ はひとつの単語と考えてください。

（注：文字列そのままではニューラルネットワークに入力できないので、なんらかの数値やベクトルなどに変換する必要があります。このトピックに関しては後述します。）

入力に対して重み $W_a$ をかけて、中間層の各列の状態 $a^{<1>}$ を得ます。そしてまた出力側の重み $W_y$ をかけて、出力 $y^{<1>}$ を得るといった一連の処理を行っています。

先ほどの翻訳の例で言うと、"Dogs are cute." の最初の単語 "Dogs" が $x^{<1>}$ に対応します。それを変換し、出力 $y^{<1>}$ は「犬」となることを期待するのです。

![chap03_rnn_1](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_rnn_1.png?raw=true)
<center>W_x -> W_a (to be replaced)</center>

上記の処理を数式で書くと以下のようになります。ただし、入力層側の活性化関数とバイアスはそれぞれ $f$ と $b_a$ 、出力層側の活性化関数とバイアスをそれぞれ $g$ と $b_y$ としています。

$$
\begin{eqnarray}
 a^{<1>}&=&f(W_a \cdot x^{<1>}+b_a) \\
 y^{<1>}&=&g(W_y \cdot a^{<1>}+b_y)
 \end{eqnarray}
 $$

もちろんこれだけで終わるのではなく、次の時刻には新しい入力が入ってくるでしょう。その場合に上記の処理を繰り返すのでは全結合ニューラルネットワークのときと変わりません。リカレントニューラルネットワークでは下図のように、その時点における入力 $x^{<2>}$ だけではなく、その1個前の中間状態 $a^{<1>}$ も重み $U_a$ をかけて加算します。

引き続き翻訳で例えると、"Dogs are cute." の "are" がここでの $x^{<2>}$ に対応します。"are" だけを見て、"are" は日本語に直すと「は」である、という推論をするのではなく、"Dogs are" を見て、「犬」に続く単語は「は」であると推論していることになります。

![chap03_rnn_2](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_rnn_2.png?raw=true)
<center>W_x -> W_a, W_a->U_a (to be replaced)</center>

数式で書くと以下のようになります。

$$
\begin{eqnarray}
 a^{<2>}&=&f(W_a \cdot x^{<2>} + U_a \cdot a^{<1>} + b_a) \\
 y^{<2>}&=&g(W_y \cdot a^{<2>}+b_y)
\end{eqnarray}
$$


それをさらに下図のように繰り返していくわけです。

![chap03_rnn_3](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_rnn_3.png?raw=true)
<center>W_x -> W_a, W_a->U_a (to be replaced)</center>

一般に、時刻 $t$ における中間層の状態 $a^{<t>}$ と出力 $y^{<t>}$ は以下のように書けます。ただし、$a^{<0>}=0$ とします。

$$
\begin{eqnarray}
a^{<t>}&=&f(W_a \cdot x^{<t>} + U_a \cdot a^{<t-1>} + b_a)\\
y^{<t>}&=&g(W_y \cdot a^{<t>}+b_y)
\end{eqnarray}
$$

このようにして、ある時刻までの時系列データの情報を中間状態にメモリのようにどんどん貯めていって、過去の状態を考慮しながら次の出力を得ていくという手法がリカレントニューラルネットワークで行っていることとなります。

このネットワークを省略して、上図の右にある形式のように記述することもあります。リカレントニューラルネットワークには、自身の中間の状態が次の時刻に再度入力されるという再帰的なループがあるので、再帰型（リカレント）と名付けられています。

### 3.3.2 RNNセル

リカレントニューラルネットワーク（以下RNN）には多数の発展系があります。RNNの発展系のうち特に有名なものに **Gated Recurrent Unit (GRU)** と **Long Short Term Memory (LSTM)** があります。これらに関して理解するための準備としてまず **RNNセル**（RNNユニットとも呼ぶ）という概念を導入しましょう。

RNNセルはさきほどの図でいうと、4つの〇で構成される部分に該当します。これをひとつのセルだと思って回路のように書き下したものが下図左になります。RNNでは、中間層における活性化関数（前節では $f$ と表していた関数）には基本的に tanh が用いられます。

![chap03_rnn_unit](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_rnn_unit.png?raw=true)


RNNセルは、ある時刻の入力 $x^{<t>}$ と一時刻前の中間層の状態 $a^{<t-1>}$ にそれぞれ重み行列をかけてバイアスを足したものを活性化関数に通す、という処理を延々と繰り返します。

$$ a^{<t>} = \tanh(W_a\cdot x^{<t>} + U_a\cdot a^{<t-1>} + b_a) $$

このセルから、たとえば何かの確率を出力したい場合には、上図右のようにSoftmaxと呼ばれる活性化関数を利用します。tanh関数の計算結果（中間層の出力）$a^{<t>}$ に
重みをかけ、バイアスを足してからSoftmaxに入力し、確率値 $y^{<t>}$ を得ます。

$$ y^{<t>} = \text{softmax}(W_y \cdot a^{<t>}+b_y)$$

このようにRNNが内部で行っている処理をひとつのセルとしてブロック状に表すと視覚的に見通しがよくなります。解きたいタスクによってどのようにセルを配置するかが変わるのですが、いくつか代表的な構成を下図に表します。

![chap03_rnn_variations](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_rnn_variations.png?raw=true)

ひとつめの "one to many" 型は例えば文字入力のアシストAIに用いられるような構成です。「お」をタイプ（入力）したら「おはようございます」という挨拶文がサジェスト（出力）されるようなイメージです。ふたつめの "many to one" 型は文書分類のタスクなどに用いられます。ニュース記事を文章を入力し、それが政治のニュースである確率を出力するようなイメージです。

みっつめの "many to many" は品詞タグ付けなどのタスクで用いられます。"I have a pen" という文章を入力したときにそれぞれの単語が「代名詞」「動詞」「冠詞」「名詞」であるということを出力するイメージです。ここまでで解説したように機械翻訳などのタスクを解くためにも用いられますが、よっつめのように元の文章を全て入力してから出力を出す、いわゆる **seq2seq** (sequence to sequence) という構成の方が性能が良いと経験的に知られています。seq2seqについては後述します。

### 3.3.3 ゲート

ここまで紹介した技術で自然言語処理のタスクを解くための準備は整ったように思えるかもしれません。しかし、もちろんそうではありません。

たとえば、"The cat which already ate ..., [was] full." という文章を考えます。猫がいろんなものをたくさん食べた結果おなかがいっぱいになったようですが、このとき [] 内の "was" を隠した状態で、カッコ内のbe動詞はどんな活用系を用いるのが正しいか当てたいとしましょう。中学校の英語のテストを思い浮かべてください。

今回の例では、主語の "cat" が単数系であり、"ate" という過去形の動詞が前にあるので正解は "was" である、というようなロジックで考えるのが自然だと思います。もし、"The cats which already ate ..., [were] full." のように "cat" が複数形 ("cats") の場合は、"were" が正解になるわけです。

この問題は（答えがbe動詞であるという事前情報があり、be動詞にかかる主語 "cat" と動詞 "ate" を特定できていれば）"cat" が単数系なのか複数形なのかと、"ate" の時制だけを見れば解けます。つまり、極端に言えばその他の単語はノイズであり、見なくて良いものです。

RNNでは入力データを平等に順々に与えていくため、基本的には「ある時点では入力を無視する」といった機能を持つことができません。同様に、入力データが長くなると中間状態に多数の情報が押し込まれ、上記の "full" を入力した段階ではすでに序盤の "The cat" の情報は「ぐちゃぐちゃ」に加工され、失われてしまいます。

そのため、情報の取捨選択を行う機構を含めたRNNの発展版が提唱されています。この取捨選択の機構は**ゲート**と呼ばれます。メモリ（中間状態）に入る扉（ゲート）の前に門番がいて、重要な情報に見える入力は通ってよし、しかしノイズは追っ払うようなイメージです。

新しい入力情報を取捨選択、つまり中間状態をアップデートするかどうかを判断するためのゲートを**アップデートゲート**と呼びましょう。

では、アップデートゲートはどのようにして実現すればよいのでしょう。アップデートするか否かというゼロイチの判断には、その値域を$(0,\,1)$の範囲に持つシグモイド関数が使えそうです。シグモイド関数の出力が $1$ (`True`) だったら中間状態を更新し、$0$ (`False`) だったら一時刻前の中間状態をそのまま用いるようにしたらいかがでしょう。

このようなロジックを実現するためのセルを下図に示します。

![chap03_gated_rnn](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_gated_rnn.png?raw=true)

内部の計算は以下の通りです。まず、素のRNNと同様に$\tanh$に通るルートがあり、その出力を $\tilde{a}^{<t>}$ とします。これと並列に、シグモイド関数を通り、出力のベクトルを $\Gamma_u$ とするルートを用意しています。

$$
\begin{eqnarray}
\tilde{a}^{<t>}&=&\tanh(W_a\cdot x^{<t>}+U_a\cdot a^{<t-1>}+b_a) \\
\Gamma_u&=&\sigma(W_u\cdot x^{<t>}+U_u\cdot a^{<t-1>}+b_u)
\end{eqnarray}
$$

そして、シグモイド関数の出力で重みをかけて、現時刻の中間状態と一時刻前の中間状態を足し合わせます。ただし、$\odot$は要素ごとの積（アダマール積）を意味するとします。

$$
a^{<t>}=\Gamma_u\odot\tilde{a}^{<t>}+(1-\Gamma_u)\odot a^{<t-1>}
$$

実際に上式に$\Gamma_u$の値を当てはめてみると「シグモイド関数の出力が$\Gamma_u=1$ (`True`) だったら中間状態を更新し、$\Gamma=0$ (`False`) だったら一時刻前の中間状態をそのまま用いる」という機構が実現できていることがわかるでしょう。


### 3.3.4 GRU と LSTM

前節で導入したゲートという概念を用いて設計されたGated Recurrent Unit (GRU) というモデルがあります。GRUは2014年に提唱されたモデルであり、前述のアップデートデート（中間状態を更新するか否かを決めるゲート）だけでなくて、今まで貯めた情報を忘れるための**リセットゲート**というものを導入しています。リセットゲートもアップデートゲートと機構としては同じであり、リセットゲートの出力ベクトル$\Gamma_r$をかけ合わせる対象が異なるだけです。$\tilde{a}^{<t>}$の計算式を書き下してみましょう。

$$
\begin{eqnarray}
\Gamma_r&=&\sigma(W_r\cdot x^{<t>}+U_r\cdot a^{<t-1>}+b_r) \\
\Gamma_u&=&\sigma(W_u\cdot x^{<t>}+U_u\cdot a^{<t-1>}+b_u) \\
\tilde{a}^{<t>}&=&\tanh(W_a\cdot x^{<t>}+U_a\cdot (\Gamma_r\odot a^{<t-1>})+b_a)
\end{eqnarray}
$$

リセットゲートの出力ベクトル$\Gamma_r$とアップデートゲートの出力ベクトル$\Gamma_u$の計算式は同じ形をしていることに注意してください。また、$\tilde{a}^{<t>}$の計算式の中で$\Gamma_r$と$a^{<t-1>}$の要素ごとの積を取っています。つまり、$\Gamma_r=0$のとき、中間状態$\tilde{a}^{<t>}$の計算にそれまでの中間状態$a^{<t-1>}$は考慮されません。言い換えると、中間状態がリセットされていることになります。

なお、以降の計算には変更はありません。

$$
a^{<t>}=\Gamma_u\odot\tilde{a}^{<t>}+(1-\Gamma_u)\odot a^{<t-1>}
$$

ゲート付きのRNNモデルには、GRUの他にもLong Short Term Memory (LSTM) と呼ばれるものもあります。LSTMはGRUよりもさらに前に考案されたモデルであり、ゲートの種類と数が異なります。また、LSTMと一口に言ってもいくつかの亜種が存在します。よく用いられるLSTMモデルは、入力ゲート、出力ゲート、そして忘却ゲートのみっつのゲートで構成されます。ここでは深く触れませんが、GRUのふたつのゲートで行われていることと似たような処理がLSTMのみっつのゲートでも行われます。（むしろ、LSTMをシンプルにしたモデルがGRUなのですが。）

GRUとLSTMのどちらが優れているかを一概にいうのは難しいです。GRUのほうがパラメーター数が少ない分、小さいデータセットに対して有利であるという報告はあります (cf. [Empirical Evaluation of Gated Recurrent Neural Networks on Sequence Modeling](https://arxiv.org/abs/1412.3555))。しかし、[On the Practical Computational Power of Finite Precision RNNs
for Language Recognition](https://arxiv.org/abs/1805.04908)ではLSTMの方がGRUよりも「厳密に強力」("strictly stronger") であるとの報告がされています。実際に利用する場合はどちらのモデルも試してみて比較するのがよいでしょう。

さて、GRUやLSTMの登場によって、シンプルなRNNよりもさらにデータの長期的な傾向を把握できるようになりました。しかし、それでもあまり長いセンテンスの文脈を掴むのは難しいようです。そのため、単純ですが強力で実用的によく用いられるトリックとしてセルを**双方向（bi-directional）**にするというものがあります。やることは本当に単純で、ふたつのLSTMなどのレイヤーを並行に配置し、例えば "dogs are cute" という文章を入力するとき、それを逆順にした "cute are dogs" という文章も並行に入力するだけです。TensorFlowのKeras APIには `tf.keras.layers.Bidirectional` というラッパーが用意されており、`tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(num_units))` のように`LSTM`などのRNN関連レイヤーをラップするだけで双方向化することができます。実際の使用例は次節で紹介します。

## 3.4 単語の数値化

ここまでリカレントニューラルネットワークの仕組みを学んできました。しかし、前述の通り、テキストの文字列をそのままニューラルネットワークに入力することはできず、なんらかの方法で文字列を数値化しなくてはいけません。本節では文字列かの数値化手法のひとつである単語の埋め込み表現について解説します。

### 3.4.1 One-Hot Encoding

本題に入る前にまず単純な手法である**One-Hot Encoding**とその問題点について考えていきましょう。

One-Hot Encodingでは文章に含まれる単語のリスト（**語彙**やvocabraryなどとも呼ぶ）を作成し、それぞれの単語を、語彙のリストと同じ長さのベクトルで表現します。それぞれの単語に対応するインデックスの値のみ1で、その他は0を詰めます。

例を見た方がわかりやすいでしょう。例えば "dogs are cute" という文と "dogs love bones" という文があったとしましょう。これらの文には dogs/are/cute/love/bones という単語が含まれています。それぞれをOne-Hot Encodingすると以下のようになります。

|     |dogs |are  |cute |love |bones|
|:---:|:---:|:---:|:---:|:---:|:---:|
|dogs |1    |0    |0    |0    |0    |
|are  |0    |1    |0    |0    |0    |
|cute |0    |0    |1    |0    |0    |

|     |dogs |are  |cute |love |bones|
|:---:|:---:|:---:|:---:|:---:|:---:|
|dogs |1    |0    |0    |0    |0    |
|love |0    |0    |0    |1    |0    |
|bones|0    |0    |0    |0    |1    |

ただし、一番左の列は文を表し、一番上の行は語彙のリストを表しています。それ以外の0と1で表された行列のことを文章の**One-Hot Encoding表現**と呼びます。余談ですが、ひとつの要素のみ0で他の全ての要素が1のビット列をOne-Coldと呼びます。

前章で画像を輝度の行列として扱ったのと同様に、One-Hot Encoding表現を用いれば文章を行列化でき、ニューラルネットワークに入力することが可能になります。

ただし、実はOne-Hot Encodingは非常に非効率的です。今回の例のように全文章中に含まれる単語の数が5つ程度であるとあまり気になりませんが、例えば単語数が10,000だとしたらいかがでしょう。単語を One-Hot Encoding するとひとつの要素のみ1で残り9,999個の要素が0のベクトルができあがってしまいます。

また、単純に単語を0と1だけのOne-Hotなベクトルにするのでは問題があります。例えば、"dog"も"bone"も同じ名詞であるとか、"are"も"love"も同じ動詞であるとか、"is"と"are"はどちらもbe動詞であり似ていると言った情報がベクトルに全く含まれません。One-Hot Encodingの場合、全ての単語のベクトルが同等に扱われてしまいます。それぞれの単語のベクトル間の「距離」が互いに等しく、単語の意味的な「距離」を扱えません。

これを解決するのが**単語の埋め込み** (Word Embedding) という手法です。

### 3.4.2 単語の埋め込み

単語の埋め込みでは、ベクトルの値を手で決めるのではなく、学習で得るパラメーターとして扱います。あらかじめ埋め込み次元の値を決めておいて、訓練を通してパラメーター更新していきます。例えば、次元数を4とすれば結果的に以下のような密な行列が得られるでしょう。なお、ここでの数値は筆者が適当に入力したもので意味はありません。

|     |dim0 |dim1 |dim2 |dim3 |
|:---:|:---:|:---:|:---:|:---:|
|dogs |0.2  |-0.1 |3.5  |1.5  |
|are  |1.8  |2.5  |-0.8 |0.4  |
|cute |-3.3 |1.1  |1.3  |-0.5 |

全結合ニューラルネットワークのそれぞれのレイヤーのユニット数はハイパーパラメーターとして調整する必要があるのと同様に、適切な埋め込み次元の大きさも扱うデータセットとタスクに依存します。

ここでは、自然言語処理の実験によく用いられる "imdb_reviews" データセットを使って単語の埋め込みとテキスト分類を行うモデルを作成してみましょう。

IMDb (Internet Moview Database) とは、Amazonの子会社であるIMDb.com, Inc.が運営する映画やテレビ番組、ゲームなどの情報を集めたデータベースです。その中から映画のレビュー文を集め、さらにそのレビューがポジティブか否かをラベル付けしたのが "imdb_reviews" データセットです。"imdb_reviews" データセットは TensorFlow Datasets から利用できます。

まずは必要なモジュールをインポートしましょう。

In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds

TensorFlow Datasetsのカタログ (https://www.tensorflow.org/datasets/catalog/imdb_reviews) を見ると同じ imdb_reviews データセットでも複数の設定で用意されているものが存在しています。今回はあらかじめ単語のリスト（語彙）を約8,000個選別してあるものを利用します。

In [2]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k', 
    split = (tfds.Split.TRAIN, tfds.Split.TEST), 
    with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incomplete94LPYZ/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incomplete94LPYZ/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incomplete94LPYZ/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


`info` (`tfds.core.DatasetInfo`) の中身を見るとそのデータセットの情報を確認できます。

In [3]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(None,), dtype=tf.int64, encoder=<SubwordTextEncoder vocab_size=8185>),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Pot

例えば、`features` 内の `'label'` を見るとクラス数 (`num_classes=2`) が確認できます。また、`features` 内の `'text'` を見ると、テキストデータが `tf.int64` 型で保存されていることがわかります。ここでは、単語からインデックスに変換するエンコーダー (`tfds.features.text.SubwordTextEncoder`) がデータ提供者よりあらかじめ用意されているので、自分で語彙リストの作成をする必要はありません。エンコーダーを取得し、単語（サブワード）のリストの一部を表示してみましょう。

In [4]:
encoder = info.features['text'].encoder
print(encoder.subwords[:20])

['the_', ', ', '. ', 'a_', 'and_', 'of_', 'to_', 's_', 'is_', 'br', 'in_', 'I_', 'that_', 'this_', 'it_', ' /><', ' />', 'was_', 'The_', 'as_']


"the" や "a" といった単語の後ろにアンダースコア "\_" がついていますが、こちらは空白文字を表しています。複数形の "s\_" がひとつの **サブワード** として登録されていることに注意してください。例えば、"apple"、"apples"、"pen"、"pens" という単語がデータセット内に含まれているとき、これら４つを別々に扱うのではなく、"apple"、"pen"、そして "s" の３つから構成されているとすることで語彙の数を節約できます。サブワード化の効果です。

さて、データセットの中身をみてみましょう。データはまず `iter()` でイテレーターを取り出し、`next()` を呼ぶことで取得できます。

In [7]:
data = next(iter(train_data))
print(data)  # tf.Tensor のタプル

(<tf.Tensor: shape=(163,), dtype=int64, numpy=
array([  62,   18,   41,  604,  927,   65,    3,  644, 7968,   21,   35,
       5096,   36,   11,   43, 2948, 5240,  102,   50,  681, 7862, 1244,
          3, 3266,   29,  122,  640,    2,   26,   14,  279,  438,   35,
         79,  349,  384,   11, 1991,    3,  492,   79,  122,  188,  117,
         33, 4047, 4531,   14,   65, 7968,    8, 1819, 3947,    3,   62,
         27,    9,   41,  577, 5044, 2629, 2552, 7193, 7961, 3642,    3,
         19,  107, 3903,  225,   85,  198,   72,    1, 1512,  738, 2347,
        102, 6245,    8,   85,  308,   79, 6936, 7961,   23, 4981, 8044,
          3, 6429, 7961, 1141, 1335, 1848, 4848,   55, 3601, 4217, 8050,
          2,    5,   59, 3831, 1484, 8040, 7974,  174, 5773,   22, 5240,
        102,   18,  247,   26,    4, 3903, 1612, 3902,  291,   11,    4,
         27,   13,   18, 4092, 4008, 7961,    6,  119,  213, 2774,    3,
         12,  258, 2306,   13,   91,   29,  171,   52,  229,    2, 1245,
    

すでにインデックス値の列としてエンコードされているため、このままではどんな文章なのか全く見当もつきません。ここで、先ほどのエンコーダーの `decode` メソッドを用います。

In [10]:
encoder.decode(data[0])

"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."

文章データをひとつひとつニューラルネットワークに入力していては非効率であるため、ミニバッチ化しましょう。ただし、文章はそれぞれ長さが異なるものなので、単純に文章を並べるだけではうまくミニバッチ化できません。そこで、帳尻を合わせるために行う操作のことを **パディング** と呼びます。例えば、"I have a pen" という４つの単語からなる文と "Dogs are cute" という３つの単語からなる文があるとしましょう。このとき、後者を "Dogs are cute `<PAD>`" のように、帳尻合わせ用のトークンをひとつ追加すれば、これら２つの文を同じ形の行列として扱うことができるようになります。

TensorFlowのデータセットを扱うときは `padded_batch` メソッドを呼ぶことで実現できます。

In [0]:
train_batches = train_data.padded_batch(batch_size=64)
test_batches = test_data.padded_batch(batch_size=64)

ここまででデータの準備はできたので、ニューラルネットワークのモデルを構築していきましょう。

いつも通り Keras の `Sequential` クラスを用いてモデルを作っていきます。初めのレイヤーには**単語の埋め込み**を行うための `Embedding` レイヤーを配置します。元々約8,000個の単語がありましたが、埋め込み空間の次元（ここでは64次元としています）に圧縮できます。

そして、`LSTM` レイヤーを `Bidirectional` でラップすることで双方向LSTMとします。その後に、画像分類のときと同様に `Dense` レイヤーを用いた分類層を構築してみます。今回はレビューがポジティブか否かの二値分類なので、出力層のユニット数は１つで活性化関数にはシグモイド関数を指定しています。

In [12]:
embedding_dim = 64

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=encoder.vocab_size, output_dim=embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64)),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=1, activation='sigmoid'),
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 598,209
Trainable params: 598,209
Non-trainable params: 0
_________________________________________________________________


二値分類なので損失関数には `binary_crossentropy` を指定します。最適化アルゴリズムには Adam を利用し、訓練中に表示するメトリクスとしては正解率 (accuracy) を用いることにしましょう。

In [14]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(
    train_batches,
    epochs=5,
    validation_data=test_batches,
    validation_steps=30
)

Epoch 1/5
391/391 [==============================] - 47s 121ms/step - loss: 0.5369 - accuracy: 0.7250 - val_loss: 0.4500 - val_accuracy: 0.8078
Epoch 2/5
391/391 [==============================] - 47s 120ms/step - loss: 0.4570 - accuracy: 0.7966 - val_loss: 0.5077 - val_accuracy: 0.7646
Epoch 3/5
391/391 [==============================] - 48s 123ms/step - loss: 0.3545 - accuracy: 0.8568 - val_loss: 0.4046 - val_accuracy: 0.8297
Epoch 4/5
391/391 [==============================] - 47s 120ms/step - loss: 0.3796 - accuracy: 0.8379 - val_loss: 0.4333 - val_accuracy: 0.8281
Epoch 5/5
391/391 [==============================] - 47s 120ms/step - loss: 0.2930 - accuracy: 0.8874 - val_loss: 0.4938 - val_accuracy: 0.8276


やや過学習していますが、バリデーションデータに対して大体80%くらいの正答率で分類を行うことができるモデルを訓練することができました。このように、テキストデータでも一度語彙リストのインデックス値に置き換え、単語の埋め込み表現に変換することで、ニューラルネットワークの入力として用いることができます。

## 3.5 Sequence to Sequence

次に、Attention機構の解説の前に、3.3.2節でも触れた **Seq2Seq** モデルを紹介します。Seq2Seq は Sequence to Sequence の略で、その形状からエンコーダー・デコーダーモデルとも呼ばれます。機械翻訳で日本語の文章を英語の文章に変換する際のように、あるシーケンスを別のシーケンスするタスクで広く用いられています。

![seq2seq](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_seq2seq.png?raw=true)

上図はSeq2Seqモデルの概要を表しています。それぞれの四角はRNNやGRU、LSTMなどのセルを表しています。例えば日本語から英語への翻訳タスクを解く場合には、図の左側のセル（エンコーダー）に対して日本語の文章をどんどん入力していきます。入力し終わったときの状態を $C$ で表しています。文脈 (Context) の頭文字を取っています。Thought Vector などと呼ばれることもあります。

このように入力文を「エンコード」した文脈ベクトルを、図の右側の「デコーダー」の中間状態の初期値として用います。そして、出力値 $y^{<t-1>}$ を時刻 $t$ におけるデコーダーへの入力として予測を行っていきます。一般的には、あらかじめ定義しておいた `<EOS>` (End of Sequence もしくは Sentence の略) タグが出力に現れたら予測終了とします。

Seq2Seqモデルは現在でもよく使われている強力なモデルではありますが、入力文が長くなるとうまく働くなってしまいます。というのも、数十の単語からなる文章を入力したとき、文脈ベクトル $C$ にはその数十の情報がひとつのベクトルに集約されることになります。

この仕組みでは文章中の単語と単語の関係などを長期にわたって覚えておくのは難しそうです。RNNの中間状態を再帰的に更新していくだけではなく、それぞれの時点での状態を適宜参照できればもっと効率的に時系列データを処理できるでしょう。これを実現する手法のひとつに **Attention 機構** (注意機構) があります。




## 3.6 Attention機構

Attention機構はその名の通り、データのどの部分に注意 (attention) を向けるかという機能をモデルに追加する仕組みのことです。例えば "Dogs are cuter than cats" という文章があり、これを日本語に翻訳したいとします。「犬は猫よりかわいい」などといった文章を出力できれば良いでしょう。

さて、最初に「犬」という単語を出力したいときに、あまり入力文の後半の単語は気にせず、下図のように "Dogs" という部分に注力して見ることができれば、最初に出力すべき日本語は「犬」だということが想像できるはずです。このように、ある程度どこに注目すべきかを重みをつけてあげるための機構が Attention と呼ばれるものです。

![attention1](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_attention1.png?raw=true)

同様に、"are" という単語は「は」に相当するとか、「猫」を出力するときは"cats" を見ておけば良いとか、"than" があってかつ前後に比較級 "cuter" があれば、「〜より」という日本語に翻訳するのが正しいだろうなどと考えられるわけです。また最後には"cuter" があるから「かわいい」と出力するのが正しいだろうという、まるで人間が英文翻訳をするのと同じような注目の仕方でそれぞれの単語の状態を見て翻訳のタスクを行うというのがAttention機構が行っていることのイメージとなります。

![attention5](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_attention5.png?raw=true)

Attention機構というのは上記のような重み付けを実現するための仕組みであり、その実装方法は多数提案されています。ここでは、先述の **Transformer** というモデルでも使われている **内積注意** (Dot Product Attention) を紹介します。

下図はSeq2Seq型のモデルに内積注意の機構を足した様子を表しています。図の左下より"He loves to eat." という文章をエンコーダーのRNNに入力していきます。そして、デコーダーのRNNよりドイツ語に翻訳した単語を出力しています。それぞれのRNNの中間状態 (Source) は１つのベクトルになります。内積注意では、それらのベクトルを並べた行列と出力部分の状態ベクトル (Target) の内積を取ります。

ちなみに内積というのは、掛け合わせる（正規化された）ベクトルが同じ方向を向いていれば $1$ で、直交していると $0$ になり、真逆の方向を向いていれば $-1$ になるような演算です。つまり、２つのベクトルの内積の大きさがそれらの関連度合い、言い換えると、ベクトル同士の類似度に相当します。

Source の状態ベクトルと Target の状態ベクトルの内積の計算をすることで、状態が近い成分を取り出すことができます。内積を取った結果をSoftmax関数に作用させたものを重み付けに用います。この重みは、検索システム内で検索クエリと検索対象のドキュメント間の類似性を表すのに用いるスコアのようなものだと考えるとイメージしやすいかもしれません。このような仕組みで、エンコーダーのどの時刻の状態に「注意」するかを決めてあげるのが内積注意です。

![dot_product_attention](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_dot_product_attention.png?raw=true)
<center>他所からの引用なので to be replaced</center>

<<< ここまでチェック >>>

### 3.6.1 Self-Attention

前節では、内積注意による入力側の状態と出力側の状態の関連性の計算について考えました。

次の図のものはもう少し複雑な説明にはなりますが、１つのベクトルに対して同じ計算を行っていくと、同じ文章の中でどの単語とどの単語が関連し合っているかを学習することができるようになります。この技術はSelf-Attention（自己注意）と呼ばれています。こちらの図はもともとミクシーの方によって書かれた書かれたものを利用させていただいていますが、たとえば、好きな動物はというような文章があったときに、このSelf-Attentionを用いて学習すると、図の〇の大きさが関連性の高さを示しているのですが、’動物’という単語と’好き’という単語の関連性が高いというような学習結果が得られます。’動物’という名詞がどういう動詞に関連性があるか、つまり単語がどの単語を修飾しているかといったことがSelf-Attentionを使うと判定できます。

![self_attention](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_self_attention.png?raw=true)
<center>他所からの引用なので to be replaced</center>

次の図は、TransformerというGoogleのニューラル機械翻訳のモデルの論文から取ってきたものになりますが、’The law will never … ‘というような文章があった場合に、図の’its’が何を指しているのかを学習することができます。’its’は代名詞なので、その前にすでに具体的な単語があってそれを指し示しているのですが、それはこの場合は’law’とつながっているとか、’its application’の’its’が何を修飾しているかというようなことを確認することができます。つまり、このSelf-Attentionというものが文章の文脈の表現を抽出するのに用いることが可能であるということをここで示しています。再度説明すると、文章つながりであったり、単語同士の関係であったりといったことを抽出することがSelf-Attentionを使うことで可能になります。

![transformer](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_transformer.png?raw=true)

![self_attention_weights](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_self_attention_weights.png?raw=true)

ここまでの説明でようやくBERTがどういうものかということが何となくイメージできるようになったのではないかと思います。これからBERTというものをもう少し説明してきます。

## 3.7 BERT

下図がBERTのモデルの構造になりますが、いままで説明したSelf-Attention機構を使って文章の文脈を獲得する部分が下図の右側の図のモデルの赤の太枠で表示されている部分にあたります。下図ではtmとして示されているこのSelf-Attention機構を図の左枠の部分のようにたくさん並べます。まず文章は時系列で左側から順番に流れるようになっていますが、それだけではなくそれぞれのSelf-Attention機構には図の左枠内の矢印のように右から左向きに文章の流れとは逆方向の接続も設けることにより、さらに強力に表現を抽出できるようにした双方向の言語表現をエンコードするモデルとなっています。

BERTを使うと何がうれしいかというと、文章の表現を学習および記述することができることです。何らかの日本語の文章で、たとえば、’神田の鬼金棒のラーメンは美味しい’のような文章が入ってきた場合に、画像の表現抽出器のように、下図の大規模データセットで訓練済みのBERTとして示されているような中間の表現抽出器を使ってある中間状態にして、それを下図のように利用することで、機械語翻訳モデル、対話モデルやこの文章がポジティブかネガティブかを調べるような感情推定モデルなどを作ったり、いろんなタスクに応用できたりします。ありがたいことに、京都大学などでWikipediaなど大規模な文章のデータセットを使って、BERTのモデルの訓練を行ってくれています。Googleが提供しているモデルには、英語、中国語など多言語対応しているものもあれば、日本語に対応したものもあります。いろんな人がやりたいようなタスクの性能をあげるためのひとつのマリオのスパワーアップキノコのような存在としてBERTを使うことができます。

以下の図はBERTが多数のベンチマークタスクの性能のリストで圧倒的な高い性能を発揮したということで話題になりました。State-Of-The-ArtとはAIのベンチマークの世界で最高性能のことを指します。たとえばSWAGというベンチマークテストはどういったものかというと、’A girl is going across a set up monkey bars. She’のような文章があった場合、それに続く文章を当てるというようなタスクになっています。選択肢としては図に示されるように4つのものがありますが、この中で一番自然なつながりがどれなのかを選ぶというものです。これはなかなか人間でも少し時間をかけて考えることで理解できるような難しい問題設定ですが、このようなものもBERTを使うとかなり高い精度で正解することができます。このような部分がBERTがすごくて驚くべきところです。

では、BERT自体はどのように学習されているかというと、Wikipediaのような文章を入力すると勝手に学習するわけではなくて、BERTに対して何かしらのテストというか宿題のようなものを用意してあげる必要があります。やっていることは下図に示す穴埋めクイズと隣接文クイズというものを解くことで、それらをBERTに学習をさせています。これはどういうものかというと、’the men went to grocery store’ というような文章があって、図のようにその中の１つの単語、この場合はgroceryの部分をマスクすることで、中にはどのような単語があったでしょうかというようなことを学ばせることをやっていたり、さきほどのSWAGのタスクと似た感じで、入力文章から次の文章がどれかを当てさせたりするようなものです。図の例の場合では、上の文章が正解なのですけれども、‘男がスーパーに行って1ガロンのミルクを買いました’というような文章になるため、文章として適切であると言えるので、ラベルにIsNexttというものが付いています。それ以外に図の下の文章のようにランダムに別のところから文章を取ってきますが、たいていランダムにとってきた場合には前後の文章としては意味が成立しないので、ラベルとしてはNotNextというものが付いてくるわけですが、実際2番目の文章には“ペンギンは鳥より飛ばない”というように隣接する文章として正しいか否かを当てるようなタスクもあります。このようなことを学習していくことで、文脈にあたる表現を獲得できるようになっていきます。

このようにして獲得したBERTのモデルに対していろいろな出力のユニットがありますが、それにアドオンでネットワークを追加してあげれば、さまざまな機能を実現でき多様なタスクに応用できるようになります。たとえば、下図の赤の点のように、分類タスク用のネットワークを追加すれば、分類ができるようになるし、文章を出力するようなネットワークを後ろに追加してあげれば翻訳などができるようになります。

## 3.8 まとめ

まとめとしては、まず時系列データに対しては、全結合ニューラルネットワークよりリカーレントニューラルネットワークの方が向いています。またリカーレントネットワークもゲートなどを追加することで単純なリカーレントニューラルネットワークの欠点を解消するこができるようになります。リカーレントネットワークの有名なモデルとしては、LSTMやGRUなどあります。機械翻訳みたいなタスクに対しては、Seq2seqのモデルが良く使われますが、その入力の文脈情報を１つのベクトルとしてまとめるのではなくて、Attentionを使ってどの文脈に注目するかを時間の関数にするというのも近年行われています。特にSelf-Attention機構はリカーレントニューラルネットワークを使っていなくても強力な文脈表現として用いることができるものです。またBERTというのは、大量のデータセットを使って事前に学習しておいてあげれば、いろんなタスクに応用できる自然言語処理のドーピング剤のような存在であります。


Synthesizer: Rethinking Self-Attention in Transformer Models
https://arxiv.org/abs/2005.00743